In [ ]:
import pandas as pd
from transformers import pipeline

sent_model = pipeline("sentiment-analysis")

c:\Users\Joman\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


c:\Users\Joman\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


## Load data

In [ ]:
df = pd.read_csv("maxdiff_dummy_data.csv")
df.head()

,video_id,text,maxdiff_mean,sample_size
0,9,Donald Trump's agenda would kill clean energy ...,0.228065,1053
1,1,Donald Trump wants to end programs that help y...,0.294940,1061
2,2,Donald Trump plans to cut infrastructure money...,0.307397,1037
3,3,Donald Trump's policy would cancel funding tha...,0.311081,999
4,31,Donald Trump's proposed policies would get rid...,0.312035,1017


## Compute weighted score

In [3]:
df['weighted_score'] = df['maxdiff_mean'] * df['sample_size']
df['weighted_score'].head()

0    240.152137
1    312.931126
2    318.770823
3    310.769706
4    317.339506
Name: weighted_score, dtype: float64

## Sentiment Analysis

In [4]:
df['sentiment'] = df['text'].apply(lambda x: sent_model(x)[0]['label'])
df['sentiment_score'] = df['text'].apply(lambda x: sent_model(x)[0]['score'])
df['sentiment'].head()

0    NEGATIVE
1    NEGATIVE
2    NEGATIVE
3    NEGATIVE
4    NEGATIVE
Name: sentiment, dtype: object

In [5]:
df['sentiment_score'].head()

0    0.953618
1    0.998914
2    0.991255
3    0.998131
4    0.995537
Name: sentiment_score, dtype: float64

## Add simple NLP features

In [6]:
df["length"] = df["text"].str.len()
df["word_count"] = df["text"].str.split().apply(len)
df["length"].head()

0    273
1    176
2    162
3    181
4    362
Name: length, dtype: int64

In [7]:
df["word_count"].head()

0    44
1    32
2    26
3    26
4    56
Name: word_count, dtype: int64

## Explore which attributes relate to higher persuasion

In [8]:
df.groupby("sentiment")["maxdiff_mean"].mean()
df[['maxdiff_mean', 'length', 'word_count']].corr()

,maxdiff_mean,length,word_count
maxdiff_mean,1.000000,-0.216372,-0.201987
length,-0.216372,1.000000,0.973497
word_count,-0.201987,0.973497,1.000000


Respondents appear to favor concise, direct language - as can be seen through the negative correlation with the `length` and `word_count` variables. This is likely because shorter messages reduce cognitive load and make the persuasive point clearer and faster to grasp.

## Top and bottom performers

In [9]:
top_texts = df.sort_values("maxdiff_mean", ascending=False).head(5)
bottom_texts = df.sort_values("maxdiff_mean", ascending=True).head(5)

top_texts[['text', 'maxdiff_mean', 'sentiment']]

,text,maxdiff_mean,sentiment
126,The Donald Trump administration gave massive t...,0.748382,NEGATIVE
125,Donald Trump's budget cuts would take away hea...,0.731315,NEGATIVE
124,"While families struggle with rising prices, Do...",0.729406,POSITIVE
123,Donald Trump's proposed budget cuts to welfare...,0.722363,NEGATIVE
122,Donald Trump's policies would force many disab...,0.719145,NEGATIVE


In [10]:
bottom_texts[['text', 'maxdiff_mean', 'sentiment']]

,text,maxdiff_mean,sentiment
0,Donald Trump's agenda would kill clean energy ...,0.228065,NEGATIVE
1,Donald Trump wants to end programs that help y...,0.294940,NEGATIVE
2,Donald Trump plans to cut infrastructure money...,0.307397,NEGATIVE
3,Donald Trump's policy would cancel funding tha...,0.311081,NEGATIVE
4,Donald Trump's proposed policies would get rid...,0.312035,NEGATIVE


Respondents were more moved by direct personal harm, and less moved by messages describing abstract or bureaucratic consequences. This suggestss that systemic impacts are less compelling than personal or human-oriented narratives.
Since all messages shared a negative framing, what mattered was how the negativity was expressed. Messages describing personal harm resonated more than messages describing technical policy impacts.

# Summary

The MaxDiff survey indicates that emotional specificity drives persuasiveness. Messages linking Trump’s policies to direct harm on real people, especially vulnerable populations, performed best. In contrast, messages that addressed broader policy mechanics, infrastructure, or programs performed noticeably worse.

Shorter, sharper messages consistently outperformed longer ones, and sentiment classification did not differentiate messages because nearly all examples had inherently negative framing. Instead, the type of negative framing (personal vs. abstract) was the key driver of persuasiveness.